### 1. Введение

✍ Мы подошли к заключительной части раздела, посвящённого математическому анализу и оптимизации. В предыдущих модулях мы уже познакомились с функциями и их исследованием, научились находить минимальные и максимальные значения для функций, а также применять эти значения для решения реальных задач. В данном модуле мы продолжим углубляться в методы оптимизации, сравним их и обсудим, для каких случаев какие алгоритмы подходят лучше всего.

За два прошлых модуля мы узнали достаточно много важных понятий и инструментов — давайте повторим их ↓

**Задание 1.1**

Пусть прибыль в вашей компании выражается следующей функцией, которая зависит от параметра x — количества производимых товаров:

$f(x)=-x^{4}+6 x^{3}-4 x^{2}+80$

Найдите максимально возможную прибыль, которую вы можете получить, варьируя количество произведённых товаров.

In [1]:
import numpy as np
import pandas as pd
import sympy
from sympy import Symbol, S
from sympy import diff
from sympy import solveset, Eq
from sympy import *
 
x= symbols('x')
f = -x**4 + 6*x**3 - 4*x**2 + 80
fa = f.diff(x) 
sol = solve(fa, x)
sol

[0, 1/2, 4]

In [2]:
fa


-4*x**3 + 18*x**2 - 8*x

In [3]:
x = 4
eval('-x**4 + 6*x**3 - 4*x**2 + 80')

144

**Задание 1.4**

Допустим, вы хотите произвести некоторое количество товара, которое зависит от часов работы двух ключевых сотрудников следующим образом:

$f(x, y)=x^{2}+2 y^{2}$

Однако вы можете оплатить этим сотрудникам не более 20 часов работы.

Какое наибольшее количество товаров вы сможете произвести в таком случае?

Помните, что количество часов работы должно быть целым, поэтому, прежде чем вычислять итоговый результат, округлите часы работы до целого.

In [4]:
from sympy import *
x,y,w = symbols(' x y w' )
g = x**2 +  2*y**2
print('Целевая функция для аргументов a и b :\n f = ', gamma)
q = x + y - 20
print('Функция ограничений: ', q,'= 0')
f = x**2 +  2*y**2 + w*(x + y - 20)
print('Функция Лагранжа :\n ',f)
fx = f.diff(x)
print('df/dx =',fx,'= 0')
fy = f.diff(y)
print('df/dy =',fy,'= 0')
fw = f.diff(w)
print('df/dw =',fw,'= 0')
sols = solve([fx,fy,fw],x,y,w)
print('Стационарная точка M(x,y):\n',float(sols[x]),',',float(sols[y]))

x = round(float(sols[x]), 0)
y = round(float(sols[y]), 0)
eval('x**2 +  2*y**2')

Целевая функция для аргументов a и b :
 f =  gamma
Функция ограничений:  x + y - 20 = 0
Функция Лагранжа :
  w*(x + y - 20) + x**2 + 2*y**2
df/dx = w + 2*x = 0
df/dy = w + 4*y = 0
df/dw = x + y - 20 = 0
Стационарная точка M(x,y):
 13.333333333333334 , 6.666666666666667


267.0

### ЭТОТ МОДУЛЬ МЫ ПОСВЯТИМ ИЗУЧЕНИЮ НОВЫХ МЕТОДОВ ОПТИМИЗАЦИИ:

- рассмотрим, какие вариации существуют у уже известного вам алгоритма градиентного спуска, и узнаем, в чём суть **обратного распространения ошибки**;
- познакомимся с **методом Ньютона** и **квазиньютоновскими методами BFGS и L-BFGS**;
- разберём область применения задач линейного программирования и попрактикуемся в их решении;
- узнаем, что такое **метод отжига** и **метод координатного спуска**.

Из предыдущего модуля вы помните, что в каких-то методах оптимизации мы использовали лишь значение функции, где-то — считали градиент, а где-то — находили матрицы производных. Эти особенности дают возможность поделить все алгоритмы на три группы:

- методы нулевого порядка (их работа основана на оценке значений самой целевой функции в разных точках);
- методы первого порядка (при работе они используют первые производные в дополнение к информации о значении функции);
- методы второго порядка (для них необходимо оценивать и значение функции, и значение градиента, и гессиан (матрицу Гессе).

**Примечание**. Иногда в литературе можно встретить термины «оракул первого порядка» или «оракул нулевого порядка». Так обозначают компоненты алгоритма, которые находят информацию на каждом шаге для метода соответствующего порядка.

В данном модуле мы будем рассматривать методы разных порядков и практически в каждом юните будем практиковаться в их применении. В конце мы обобщим полученные знания и сравним, какие методы и в каких случаях показывают наилучшие результаты.

→ Важно отметить, что некоторые алгоритмы оптимизации мы пока не сможем использовать по прямому назначению: мы опробуем их на известных вам моделях линейной или логистической регрессии или на функциях одной или нескольких переменных, но основная сфера их применения — это искусственные нейронные сети. Тем не менее, понимание всего пула алгоритмов поможет вам в будущем без проблем подбирать необходимый метод вне зависимости от того, задачу какой сложности вы решаете: минимизируете простейшую функцию или обучаете нейронную сеть со сложной архитектурой.

___________________________

### 2. Градиентный спуск: применение и модификации

✍ В предыдущем модуле мы познакомились с алгоритмом градиентного спуска, а также с его очень популярной вариацией — градиентным спуском с momentum. Но на самом деле у градиентного спуска очень много модификаций, и, поскольку это действительно самый известный алгоритм, он является основой множества методов оптимизации. В этом юните мы рассмотрим и сравним три вариации градиентного спуска, которые используются наиболее часто.

Чтобы лучше понимать, какую роль играют методы оптимизации в DS и почему их так много, важно понимать основные **принципы работы нейронных сетей**. Также в современных статьях и обзорах разных методов оптимизации они практически всегда рассматриваются в контексте применения в нейронных сетях.

Процесс обучения человеческого мозга очень сложен, и наука пока не может дать достаточно подробный ответ на вопрос о том, как мы получаем и усваиваем знания, как принимаем решения. Однако той информации, которую уже удалось получить, оказалось достаточно, чтобы по аналогии создать модели искусственных нейронных сетей.

Люди учатся через пробы и ошибки, через процесс **синаптической пластичности**. Точно так же, как связи в мозге укрепляются и формируются по мере того, как мы переживаем новые события, мы обучаем искусственные нейронные сети, вычисляя ошибки нейросетевых предсказаний и усиливая или ослабляя внутренние связи между нейронами на основе этих ошибок.

Для лучшего понимания удобно воспринимать нейронную сеть как функцию, которая принимает входные данные для получения итогового прогноза. Переменными этой функции являются параметры, или веса, нейрона.

Следовательно, ключевым моментом для решения задачи, которую мы ставим для нейронной сети, будет корректировка значений весов таким образом, чтобы они аппроксимировали или наилучшим образом представляли набор данных.

На изображении ниже показана простая нейронная сеть, которая получает входные данные $(X_1, \ X_2, \  X_3, \ X_n)$. Эти входные данные передаются нейронам в слое, содержащем веса $(W_1, \ W_2, \ W_3, \ W_n)$. Входные данные и веса подвергаются операции умножения, и результат суммируется с помощью специальной функции, а функция активации регулирует конечный результат модели.

<img src="data\MATHML_md6_2_1.png" alt="drawing" width="700"/>

Для того чтобы оценить, насколько эффективно работает наша нейронная сеть, необходим показатель оценки разницы между предсказанием нейронной сети и фактическим значением целевой функции, позволяющий корректировать параметры сети так, чтобы разница между прогнозом и реальностью была как можно меньше. В Data Science эту разницу часто называют **функцией стоимости**.

Ниже представлена модель работы простейшей нейронной сети:

<img src="data\MATHML_md6_2_2.png" alt="drawing" width="700"/>

На этом изображении мы можем видеть модель простой нейронной сети из плотно связанных нейронов, которая классифицирует рукописные представления цифр 0, 1, 2, 3. Каждый нейрон в выходном слое соответствует цифре. Чем выше активация соединения с нейроном, тем выше вероятность, выдаваемая нейроном. Вероятность соответствует вероятности того, что цифра, переданная вперёд по сети, связана с активированным нейроном.

Когда цифра 3 передаётся через сеть, мы ожидаем, что соединения (представленные на диаграмме стрелками), ответственные за классификацию этой цифры, будут иметь более высокую активацию, что приводит к более высокой вероятности для выходного нейрона, связанного с цифрой 3.

Если объяснить происходящее более простыми словами, то механизм работы примерно такой:

1. На первом слое мы выделяем из цифры какие-то первичные признаки (округлости, палочки).
2. На втором слое мы выделяем уже какие-то паттерны (фрагменты цифры).
3. На третьем слое паттерны складываются в целую цифру и мы можем предсказать результат.
   
Каждый раз элементы, более похожие на элементы цифры 3, дают более высокую активацию.

За активацию нейрона отвечает несколько компонентов, и мы должны менять их в процессе обучения нашей нейронной сети, повышая качество её предсказания.

Допустим, мы используем уже хорошо известную вам среднеквадратичную ошибку как меру для оценки качества модели. В этом случае мы каждый раз вычисляем её, получаем результат и отправляем его обратно для коррекции весов в сети. Здесь как раз и появляется понятие **обратного распространения ошибки**.

**Обратное распространение (backpropagation)** — это механизм, с помощью которого компоненты, влияющие на итоговый результат, итеративно корректируются для уменьшения функции стоимости.

Важнейшим математическим процессом, связанным с обратным распространением, является вычисление производных. Операции обратного распространения вычисляют частную производную функции стоимости по отношению к весам и активациям предыдущего слоя, чтобы определить, какие значения влияют на градиент функции стоимости.

→ В процессе минимизации  функции ошибки мы постоянно вычисляем значение градиентов и таким образом приходим к локальному минимуму. На каждом этапе обучения нейронной сети её веса пересчитываются с помощью найденного градиента, причём скорость обучения (или, мы уже называли её ранее, темп обучения или шаг градиентного спуска) определяет коэффициент, с которым вносятся изменения в значения весов. Это повторяется на каждом шаге обучения нейронной сети. Нашей целью является постоянное приближение к локальному минимуму.

Принцип обратного распространения ошибки заключается в том, что сначала мы устанавливаем какие-то случайные веса (параметры) для модели, находим итоговую ошибку и движемся по сети обратно, корректируя веса (для этого вычисляем частные производные, т. е. градиент).

Данный процесс выглядит следующим образом:

<img src="data\MATHML_md6_2_3.gif" alt="drawing" width="400"/>

Распространение ошибок и использование частных производных для корректировки весов происходит до тех пор, пока не будут скорректированы все параметры в сети вплоть до первого слоя.

Если вам хочется больше узнать про механизм обратного распространения ошибки, рекомендуем обратиться к этой статье https://brilliant.org/wiki/backpropagation/.

Как вы можете видеть, даже в самом простом примере нейронной сети много переменных и действий, которые с ними происходят. Из-за этого ландшафт функции потерь для нейронных сетей становится очень сложным. К примеру, ландшафт для нейронной сети с 56 слоями может выглядеть так:

<img src="data\MATHML_md6_2_4.png" alt="drawing" width="500"/>

В предыдущем модуле мы без проблем решили задачу с помощью классического градиентного спуска, и результат совпал с итогом, полученным с помощью реализации алгоритма в стандартной библиотеке Python. Но функция потерь там была совершенно другой. Когда дело доходит до искусственных нейронных сетей, алгоритмы оптимизации сталкиваются с множеством проблем.

**Основные проблемы при реализации градиентного спуска**:

- Классический градиентный спуск склонен застревать в точках локального минимума и даже в седловых точках, словом — везде, где градиент равен нулю. Это мешает найти глобальный минимум.
- Обычно у оптимизируемой функции очень сложный ландшафт: где-то она совсем пологая, где-то более крутой обрыв. В таких ситуациях градиентный спуск показывает не лучшие результаты. Так происходит потому, что в алгоритме градиентного спуска фиксированный шаг, а нам в идеале хотелось бы его изменять в зависимости от формы функции прямо в процессе обучения.
- Много проблем возникает из-за темпа обучения: при низком алгоритм сходится невероятно медленно, при быстром — «пролетает» мимо минимумов.
- При обучении градиентного спуска координаты в некоторых измерениях могут редко изменяться, что приводит к плохой обобщающей способности алгоритма. Можно попытаться придать каждого признаку бόльшую важность, но в таком случае есть серьёзный риск переобучить модель.
  
Тем не менее, нельзя отрицать, что градиентный спуск — невероятно эффективный и популярный алгоритм. Допустим, он прекрасно подходит для минимизации среднеквадратичной ошибки в случае решения задачи регрессии. Однако в силу его несовершенств, которые очень явно проявляются в сложных моделях (например, в нейронных сетях), были созданы некоторые его модификации, которые позволяют решать задачи с большей результативностью.

Обычно выделяют **три основных вариации градиентного спуска**:

- Batch Gradient Descent;
- Stochastic Gradient Descent;
- Mini-batch Gradient Descent.
  
Далее мы рассмотрим все эти модификации и обсудим их различия. Важно понимать, что, несмотря на то что про их применение часто говорят именно в контексте нейронных сетей, они прекрасно подходят и для использования с обычными методами машинного обучения. К примеру, для стандартной линейной регрессии также подходит и обычный градиентный спуск.

### BATCH GRADIENT DESCENT

Первая вариация — это **Batch Gradient Descent**. По-русски её называют **пакетным градиентным спуском**, или **ванильным градиентным спуском** (хотя англоязычную вариацию Vanilla Gradient Descent чаще не переводят). По сути, это и есть классический градиентный спуск, который мы с вами рассматривали в предыдущем модуле.

<img src="data\MATHML_md6_2_5.png" alt="drawing" width="700"/>

Часто в различных источниках шаг Batch Gradient Descent записывается в следующих обозначениях:

$\theta=\theta-\eta \cdot \nabla_{\theta} J(\theta)$

**Примечание**. На самом деле совершенно не важно, какими буквами выражать те или иные объекты, но мы будем использовать в этом модуле обозначения, которые часто встречаются в научных статьях и различной литературе.

Здесь $\theta$ — вектор с параметрами функции, $\eta$ — шаг градиента, $\nabla_{\theta} J(\theta)$ — градиент функции, найденный по её параметрам.

Таким образом, на каждом шаге градиентный спуск находит направление наискорейшего убывания функции и движется чётко по нему. Поэтому для несложных функций он достаточно быстро сходится. Его обучение можно изобразить следующим образом:

<img src="data\MATHML_md6_2_6.png" alt="drawing" width="600"/>

Такой градиентный спуск достаточно хорошо работает, если мы рассматриваем выпуклые или относительно гладкие функции ошибки. Такие, к примеру, мы можем наблюдать у линейной или логистической регрессии. Мы обсуждали, что градиентный спуск не умеет находить глобальный минимум среди прочих, но, например, для выпуклой функции он может это сделать. Поэтому с выпуклыми функциями (допустим, со среднеквадратичной ошибкой для линейной регрессии) нам удобнее всего использовать именно его.

Но всё усложняется, когда мы хотим применить его при обучении нейронных сетей. Как уже говорилось, у таких моделей функция потерь имеет много локальных экстремумов, в каждом из которых градиентный спуск может застрять. Когда данных очень много, а оптимизируемая функция очень сложная, данный алгоритм применять затруднительно, так как поиск градиента по всем наблюдениям делает задачу очень затратной в плане вычислительных ресурсов.

### STOCHASTIC GRADIENT DESCENT

Представим, что мы реализуем градиентный спуск для набора данных объёмом 10 000 наблюдений и у нас десять переменных. Среднеквадратичную ошибку считаем по всем точкам, то есть для 10 000 наблюдений. Производную необходимо посчитать по каждому параметру, поэтому фактически за каждую итерацию мы будем выполнять не менее 100 000 вычислений. И если, допустим, у нас 1000 итераций, то нам нужно 100000*1000=100000000 вычислений. Это довольно много, поэтому градиентный спуск на сложных моделях и при использовании больших наборов данных работает крайне долго.

Чтобы преодолеть эту проблему, придумали **стохастический градиентный спуск**. Слово «стохастический» можно воспринимать как синоним слова «случайный». Где же при использовании градиентного спуска может возникнуть случайность? При выборе данных. При реализации стохастического спуска вычисляются градиенты не для всей выборки, а только для случайно выбранной единственной точки.

<img src="data\MATHML_md6_2_7.png" alt="drawing" width="700"/>

Это значительно сокращает вычислительные затраты.

В виде формулы это можно записать следующим образом:

$\theta=\theta-\eta \cdot \nabla_{\theta} J\left(\theta ; x^{(i)} ; y^{(i)}\right)$

На визуальном представлении ниже можно увидеть, что стохастический спуск создаёт много колебаний при сходимости. Это происходит как раз за счёт того, что берётся не вся выборка, а только один объект, и между объектами может быть достаточно большая разница. Чем меньше выборка, тем меньше стабильности при реализации.

<img src="data\MATHML_md6_2_8.png" alt="drawing" width="600"/>

Стохастический градиентный спуск очень часто используется в нейронных сетях и сокращает время машинных вычислений, одновременно повышая сложность и производительность крупномасштабных задач.

### MINI-BATCH GRADIENT DESCENT

Третья вариация градиентного спуска — **Mini-batch Gradient Descent**. Также можно называть его м**ини-пакетным градиентным спуском**. По сути, эта модификация сочетает в себе лучшее от классической реализации и стохастического варианта. На данный момент это наиболее популярная реализация градиентного спуска, которая используется в глубоком обучении (т. е. в обучении нейронных сетей).

В ходе обучения модели с помощью мини-пакетного градиентного спуска обучающая выборка разбивается на пакеты (батчи), для которых рассчитывается ошибка модели и пересчитываются веса.

<img src="data\MATHML_md6_2_9.png" alt="drawing" width="700"/>

То есть, с одной стороны, мы используем все преимущества обычного градиентного спуска, а с другой — уменьшаем сложность вычислений и повышаем их скорость по аналогии со стохастическим спуском. Кроме того, алгоритм работает ещё быстрее за счёт возможности применения векторизованных вычислений.

Формализовать это можно следующим образом:

$\theta=\theta-\eta \cdot \nabla_{\theta} J\left(\theta ; x^{(i: i+n)} ; y^{(i: i+n)}\right)$

Как показывает визуализация ниже, амплитуда колебаний при сходимости алгоритма больше, чем в классическом градиентном спуске, но меньше, чем в стохастическом:

<img src="data\MATHML_md6_2_10.png" alt="drawing" width="600"/>

**Резюмируем сравнение трёх вариаций градиентного спуска**:

<img src="data\pic-1.png" alt="drawing" width="1100"/>

✍ Итак, мы рассмотрели алгоритмы градиентного спуска и сравнили их параметры. Теперь, если вам необходимо будет выбрать метод градиентного спуска, вы сможете понять, какой из них подходит лучше всего.

**Задание 2.7**

Давайте потренируемся применять стохастический градиентный спуск для решения задачи линейной регрессии. Мы уже рассмотрели его реализацию «с нуля», однако для решения практических задач можно использовать готовые библиотеки.

Загрузите стандартный датасет об алмазах из библиотеки Seaborn:

In [5]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
df = sns.load_dataset('diamonds')

Удалите часть признаков:

In [6]:
df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)

Закодируйте категориальные признаки:

In [7]:
df = pd.get_dummies(df, drop_first=True)

Логарифмируйте признаки:

In [8]:
df['carat'] = np.log(1+df['carat'])
df['price'] = np.log(1+df['price'])

Определите целевую переменную и предикторы:

In [9]:
X = df.drop(columns="price")
y = df["price"]

Разделите выборку на обучающую и тестовую (объём тестовой возьмите равным 0.33), значение random_state должно быть равно 42.

Теперь реализуйте алгоритм линейной регрессии со стохастическим градиентным спуском (класс SGDRegressor). Отберите с помощью GridSearchCV оптимальные параметры по следующей сетке:

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
parameters = {
"loss": ["squared_error", "epsilon_insensitive"],
"penalty": ["elasticnet"],
"alpha": np.logspace(-3, 3, 15),
"l1_ratio": np.linspace(0, 1, 11),
"max_iter": np.logspace(0, 3, 10).astype(int),
"random_state": [42],
"learning_rate": ["constant"],
"eta0": np.logspace(-4, -1, 4)
}

Обучите регрессию с оптимальными параметрами. В качестве ответа введите получившееся значение MSE, предварительно округлив его до третьего знака после точки-разделителя.

In [11]:
sgd = SGDRegressor(random_state=42)
sgd_cv = GridSearchCV(estimator=sgd, param_grid=parameters, n_jobs=-1)
sgd_cv.fit(X_train, y_train)
print(sgd_cv.best_params_)
sgd = SGDRegressor(**sgd_cv.best_params_)
sgd.fit(X_train, y_train)
sgd.score(X_train, y_train) # r2
ls = sgd.predict(X_test)
round(mean_squared_error(y_test, ls), 3)

{'alpha': 0.0026826957952797246, 'eta0': 0.001, 'l1_ratio': 0.0, 'learning_rate': 'constant', 'loss': 'epsilon_insensitive', 'max_iter': 21, 'penalty': 'elasticnet', 'random_state': 42}


0.043

### *БОНУС: АЛГОРИТМЫ, ОСНОВАННЫЕ НА ГРАДИЕНТНОМ СПУСКЕ

Градиентный спуск настолько популярен и хорошо применим для решения различных задач, что послужил основой множества дополнительных методов. Поговорим про некоторые из них.

Иногда в данных присутствуют очень редко встречающиеся входные параметры.

Например, если мы классифицируем письма на «Спам» и «Не спам», таким параметром может быть очень специфическое слово, которое встречается в спаме намного реже других слов-индикаторов. Или, если мы говорим о распознавании изображений, это может быть какая-то очень редкая характеристика объекта.

В таком случае нам хотелось бы иметь для каждого параметра свою скорость обучения: чтобы для часто встречающихся она была низкой (для более точной настройки), а для совсем редких — высокой (это повысит скорость сходимости). То есть нам очень важно уметь обновлять параметры модели, учитывая то, насколько типичные и значимые признаки они кодируют.

Решение этой задачи предложено в рамках алгоритма **AdaGrad** (его название обозначает, что это адаптированный градиентный спуск). В нём обновления происходят по следующему принципу:

$G_t = G_t + g^2_t$
$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{G_t + \epsilon}} g_t$

Здесь мы храним сумму квадратов градиентов для каждого параметра. Таким образом, параметры, которые сильно обновляются каждый раз, начинают обновляться слабее. Скорость обучения в таком алгоритме будет постоянно затухать. Мы будем начинать с больших шагов, и с приближением к точке минимума шаги будут уменьшаться — это улучшит скорость сходимости.

Данный алгоритм достаточно популярен и работает лучше стохастического градиентного спуска. Его использует и компания Google в своих алгоритмах классификации изображений.

Однако снижение скорости обучения в AdaGrad иногда происходит слишком радикально, и она практически обнуляется. Чтобы решить эту проблему, были созданы алгоритмы RMSProp, AdaDelta, Adam и некоторые другие.

Если вам интересно подробнее узнать о перечисленных алгоритмах и окунуться в процесс оптимизации нейронных сетей, рекомендуем обратиться к следующим статьям:

- "An overview of gradient descent optimization algorithms" https://ruder.io/optimizing-gradient-descent/index.html#adagrad  
- «Методы оптимизации нейронных сетей» https://habr.com/ru/post/318970/  

________________________________________

### 3. Метод Ньютона

✍ В предыдущем юните мы познакомились с различными вариациями градиентного спуска. Но для того чтобы наш арсенал методов был более полным и позволял решать самые разные задачи, нам необходимо разобраться и с рядом других алгоритмов. В этом юните мы будем изучать метод Ньютона.

Метод Ньютона используется во многих алгоритмах машинного обучения. Часто в литературе его сравнивают с градиентным спуском, так как два этих алгоритма очень популярны. Вы уже сталкивались с методом Ньютона, но не знали об этом.

В документации https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html для функции LogisticRegression из библиотеки scikit-learn представлено пять вариантов алгоритмов оптимизации, которые можно использовать при обучении модели:

- 'newton-cg';
- 'lbfgs';
- 'liblinear';
- 'sag';
- 'saga'.

Последние два являются вариациями стохастического градиентного спуска (а значит вам уже понятен принцип их работы), а с первыми тремя нам только предстоит познакомиться. В этом юните мы рассмотрим алгоритм 'newton-cg', в следующем — 'lbfgs', а в седьмом юните — 'liblinear'. Вы будете понимать суть всех методов, представленных в самой популярной библиотеке для машинного обучения, и выбирать подходящий, исходя из особенностей поставленной задачи.

Начнём с метода Ньютона. Этот алгоритм работает быстрее, чем градиентный спуск, и тратит меньше времени для достижения минимума, однако у него есть и определённые недостатки, о которых мы поговорим позже.

Метод Ньютона изначально появился как метод решения уравнений вида $f(x) = 0$.

Проиллюстрируем принцип его работы геометрически. Пусть у нас есть график некоторой функции. Проведём к нему касательную в точке $x_n$.

<img src="data\MATHML_md6_3_1.png" alt="drawing" width="700"/>

Тогда эта касательная имеет наклон, равный $f'(x_n)$, и проходит через точку $x_n$, $f(x_n)$. В таком случае мы можем сказать, что уравнение этой касательной: $y = f'(x_n) (x - x_n) + f(x_n)$.

Так как нам необходимо решить уравнение, то нужно попасть в такую точку $x_{n+1}$, чтобы в ней значение координаты по оси ординат было нулевым, то есть в точку с координатами $x =  x_{n+1}$ и $y = 0$.

Подставим это в наше уравнение касательной:

$y=f^{\prime}\left(x_{n}\right)\left(x-x_{n}\right)+f\left(x_{n}\right)$  
$x=x_{n+1}, y=0$  
$0=f^{\prime}\left(x_{n}\right) \cdot\left(x_{n+1}-x_{n}\right)+f\left(x_{n}\right)$  
$f^{\prime}\left(x_{n}\right) \cdot\left(x_{n+1}-x_{n}\right)=-f\left(x_{n}\right)$  
$x_{n+1}-x_{n}=-\frac{f\left(x_{n}\right)}{f^{\prime}\left(x_{n}\right)}$  
$x_{n+1}=x_{n}{-\frac{f\left(x_{n}\right)}{f^{\prime}\left(x_{n}\right)}}$  

Можно посмотреть на это и в анимации:

<img src="data\MATHML_md6_3_2.gif" alt="drawing" width="600"/>

Мы видим, как для $x$ вычисляется $f(x)$, строится касательная, и в точке пересечения касательной с осью $Ox$ строится новая точка, к которой также строится касательная, и так далее. Математически доказано, что таким образом $x_i$ приближается к значению, где $f(x) = 0$. 

Формально первый  шаг этого алгоритма мы можем записать следующим образом:

$x_1 = x_0 - \frac{f(x_0)}{f'(x_0)}$   

Все остальные шаги можно обобщить с помощью следующей зависимости:

$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$  

Шаги могут повторяться сколько угодно раз до достижения необходимой точности.

Давайте посмотрим, как с использованием этого метода можно решить уравнение ↓

**Пример № 1**

Найти корень уравнения $x^2 - 4x - 7 = 0$, который находится рядом с точкой  $x = 5$, с точностью до тысячных.

Функция: $f(x) = x^2 - 4x - 7 = 0$  
Начальная точка: $x_0 = 5$  
Производная для функции: $f'(x) = 2x - 4$ 

Начнём поочерёдно совершать шаги и переходить в следующие точки, используя формулу, которую мы рассмотрели ранее:

$x_1 = 5 - \frac{5^2 - 4 \times 5 - 7}{2 \times 5 - 4} = 5 - \frac{(-2)}{6}  = \frac{16}{3} \approx 5.33333$  
$x_2 = \frac{16}{3} - \frac{(\frac{16}{3})^2 - 4 (\frac{16}{3}) - 7}{2 (\frac{16}{3}) - 4} = \frac{16}{3} - \frac{\frac{1}{9}}{\frac{20}{3}} = \frac{16}{3} - \frac{1}{60} = \frac{319}{60} \approx 5.31667$  
$x_3 = \frac{319}{60} - \frac{(\frac{319}{60})^2 - 4 (\frac{319}{60}) - 7}{2 (\frac{319}{60}) - 4} = \frac{319}{60} - \frac{\frac{1}{3600}}{\frac{398}{60}} \approx 5.31662$  

Мы видим, что в последних двух точках мы уже находимся примерно в одном и том же месте. На всякий случай проверим ещё одну:

$x_4 = 5.31662 - \frac{(5.31662)^2 - 4 (5.31662) - 7}{2 (5.31662) - 4} = 5.31662$

Действительно, мы нашли точку с точностью до тысячных, в которой останемся — это и будет нашим ответом.

**? Пример № 2**

Найти корни сложного полинома $f(x) = 6x^5 - 5x^4 - 4x^3 + 3x^2$.

Как мы знаем, к сожалению, для полинома пятой степени нет формулы поиска корней, поэтому будем использовать численные методы. В этих случаях приходится прибегать к числовому линейному приближению.

Ниже представлен график нашего полинома. У него три корня: в точках 0, 1 и где-то между ними. Как найти третий корень?

<img src="data\MATHML_md6_3_3.png" alt="drawing" width="500"/>

В методе Ньютона мы берём случайную точку $x_0$, а затем проводим касательную в ней. Точка $x_1$, где эта касательная пересекает ось абсцисс, станет нашим следующим предположением. Так что теперь мы  строим уже касательную в этой точке, и так далее . Мы продолжаем до тех пор, пока не достигнем необходимой точности. В целом, мы можем сделать приближение настолько близким к нулю, насколько хотим.

**Задание 3.1**

Найдите третий корень полинома $f(x) = 6x^5 - 5x^4 - 4x^3 + 3x^2$, взяв за точку старта 0.7. Введите получившееся значение с точностью до трёх знаков после точки-разделителя.

Попробуйте реализовать алгоритм с использованием Python на основе алгоритма градиентного спуска, изученного в предыдущем модуле.

In [12]:
def func1(x):
    return 6*x**5-5*x**4-4*x**3+3*x**2

def func2(x):
    return 30*x**4-20*x**3-12*x**2+6*x

init_value = 0.7
iter_count = 0
x_curr = init_value
epsilon = 0.000001
f = func1(x_curr)

while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
print(x_curr)
print(iter_count)

0.6286669787764609
4


? Отлично, мы научились искать приближённые значения для корней уравнения. Но как же это поможет нам найти минимум или максимум для функции?

Дело в том, что в задаче оптимизации можно решать не $f(x) = 0$, а $f'(x) = 0$ — тогда мы найдём потенциальные точки экстремума.

В многомерном случае по аналогичным рассуждениям производная превращается в градиент, а вторая производная превращается в гессиан (матрица вторых производных или, как мы её называли в предыдущем модуле, матрица Гессе). Поэтому в формуле появится обратная матрица.

Для многомерного случая формула выглядит следующим образом:

$x^{(n+1)} = x^{(n)} - \left [Hf(x^{(n)})  \right ]^{-1} \nabla f(x^{(n)})$  

Можно заметить, что эта формула совпадает с формулой для градиентного спуска, но вместо умножения на learning rate (темп обучения) используется умножение на обратную матрицу к гессиану. Благодаря этому функция может сходиться за меньшее количество итераций, так как мы учитываем информацию о выпуклости функции через гессиан. Можно увидеть это на иллюстрации работы двух методов, где метод Ньютона явно сходится быстрее:

<img src="data\MATHML_md6_3_4.png" alt="drawing" width="900"/>

Метод Ньютона, если считать в количестве итераций, в многомерном случае (с гессианом) работает быстрее градиентного спуска.

Выше мы уже разобрали применение метода Ньютона для поиска корней уравнения. Теперь давайте снова используем его, но уже для оптимизации функции ↓

? Оптимизировать функцию $f(x) = x^3 - 3x^2 -45x  + 40$.

Находим производную функции:

$f^{\prime}=3 x^{2} - 6x - 45$

Находим вторую производную:

$f^{\prime \prime}=6x - 6$

Сразу определим их в Python, чтобы можно было параллельно решить задачу и с помощью программирования:

In [13]:
def func1(x):
    return 3*x**2 - 6*x -45
def func2(x):
    return 6*x - 6

Теперь необходимо взять какую-нибудь изначальную точку. Например, пусть это будет точка x = 42. Также нам необходима точность — её возьмем равной 0.0001. На каждом шаге будем переходить в следующую точку по уже упомянутой выше формуле:

$x^{(n+1)}=x^{(n)}-\frac{f^{\prime}\left(x^{(n)}\right)}{f^{\prime \prime}\left(x^{(n)}\right)}$

Например, в нашем случае следующая после 42 точка будет рассчитываться следующим образом:

$x_{2}=42-\frac{f^{\prime}\left(x_{1}\right)}{f^{\prime \prime}\left(x_{2}\right)}$  
$f^{\prime}\left(x_{1}\right)=3 x^{2}-6 x-45 \mid _{x_{1}=42}\;= 3 \cdot 42^{2}-6 \cdot 42-45=4995$  
$f^{\prime \prime}\left(x_{1}\right)=6 x-6 \mid _{x_{1}=42} \; = \;6.42-6=246$  
$x_{2}=42-\frac{4995}{246} \approx 42-20.305=21.695$

Третья точка будет вычисляться по аналогичному принципу:

$x_3 = 21.695 - \frac{f'(21.695)}{f''(21.695)}$

Но, к счастью, нам совсем не обязательно высчитывать всё вручную — воспользуемся Python и распишем наш алгоритм:

In [14]:
initial_value = 42
iter_count = 0
x_curr = initial_value
epsilon = 0.0001
f = func1(x_curr)

while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)

#21.695121951219512
#11.734125501243229
#7.1123493600499685
#5.365000391507974
#5.015260627016227
#5.000029000201801
#5.000000000105126
#5.0

21.695121951219512
11.734125501243229
7.1123493600499685
5.365000391507974
5.015260627016227
5.000029000201801
5.000000000105126
5.000000000000001


Можно объединить всё в одну функцию:

In [15]:
def newtons_method(f, der, eps, init):
    iter_count = 0
    x_curr = init
    f = f(x_curr)
    while (abs(f) > eps):
        f = f(x_curr)
        f_der = der(x_curr)
        x_curr = x_curr - (f)/(f_prime)
        iter_count += 1
    return x_curr
 
from scipy.optimize import newton
newton(func=func1,x0=50,fprime=func2, tol=0.0001)

5.0

У метода Ньютона есть **ряд достоинств и недостатков**.

<img src="data\happy-icon.png" alt="drawing" width="70"/>

- Если мы минимизируем квадратичную функцию, то с помощью метода Ньютона можно попасть в минимум целевой функции за один шаг.

- Также этот алгоритм сходится за один шаг, если в качестве минимизируемой функции выступает функция из класса поверхностей вращения (т. е. такая, у которой есть симметрия).

- Для несимметричной функции метод не может обеспечить сходимость, однако скорость сходимости  всё равно превышает скорость методов, основанных на градиентном спуске.

<img src="data\sad-icon.png" alt="drawing" width="70"/>

- Этот метод очень чувствителен к изначальным условиям.

При использовании градиентного спуска мы всегда гарантированно движемся по антиградиенту в сторону минимума. В методе Ньютона происходит подгонка параболоида к локальной кривизне, и затем алгоритм движется к неподвижной точке данного параболоида. Из-за этого мы можем попасть в максимум или седловую точку. Особенно ярко это видно на невыпуклых функциях с большим количеством переменных, так как у таких функций седловые точки встречаются намного чаще экстремумов.

Поэтому здесь необходимо обозначить ограничение: метод Ньютона стоит применять только для задач, в которых целевая функция выпуклая.

Впрочем, это не является проблемой. В линейной регрессии или при решении задачи классификации с помощью метода опорных векторов или логистической регрессии мы как раз ищем минимум у выпуклой целевой функции, то есть данный алгоритм подходит нам во многих случаях.


- Также метод Ньютона может быть затратным с точки зрения вычислительной сложности, так как требует вычисления не только градиента, но и гессиана и обратного гессиана (при делении на матрицу необходимо искать обратную матрицу).

Если у задачи много параметров, то расходы на память и время вычислений становятся астрономическими. Например, при наличии 50 параметров нужно вычислять более 1000 значений на каждом шаге, а затем предстоит ещё более 500 операций нахождения обратной матрицы. Однако метод всё равно используют, так как выгода от быстрой сходимости перевешивает затраты на вычисления.

Несмотря на его ограниченное практическое применение, метод Ньютона по-прежнему представляет большую ценность. Он имеет большое преимущество перед градиентным спуском в силу своей быстроты и отсутствия необходимости в настройке гиперпараметра шага (мы помним, что в градиентном спуске выбор шага — довольно непростая задача, а здесь можно обойтись без этого). Причём преимущество в быстроте очень ощутимое: в сравнении на реальных данных метод Ньютона находит решение задачи за 3 итерации, а градиентный спуск — за 489. То есть мы сильно выигрываем в скорости сходимости, а для анализа данных это очень важно, ведь экономия времени и вычислительных ресурсов позволяет решать задачи быстрее.

? Мы увидели, какой эффективной может быть оптимизация второго порядка при правильном использовании. Но что, если бы мы могли каким-то образом использовать эффективность, полученную при использовании производных второго порядка, но при этом избежать вычислительных затрат на вычисление обратного гессиана? Другими словами — можем ли мы создать алгоритм, который будет своего рода гибридом между градиентным спуском и методом Ньютона, где мы сможем получать более быструю сходимость, чем градиентный спуск, но меньшие вычислительные затраты на каждую итерацию, чем в методе Ньютона?

Оказывается, такой алгоритм существует. Точнее, целый класс таких методов оптимизации, называемых **квазиньютоновскими методами**. Мы познакомимся с ними уже в следующем юните, но для начала давайте закрепим пройденный материал ↓

**Задание 3.6**

Дана функция $f(x) = x^3 - 72x - 220$. Найдите решение уравнения $f(x) = 0$ для поиска корня в окрестностях точки $x_0 = 12$. Ответ округлите до трёх знаков после точки-разделителя.

In [16]:
def func1(x):
    return x**3-72*x - 220

def func2(x):
    return 3*x**2-72

init_value = 12
iter_count = 0
x_curr = init_value
epsilon = 0.000001
f = func1(x_curr)

while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
print(x_curr)
print(iter_count)

9.727134419408875
5


$x_1 = x_0 - \frac{f(x_0)}{f'(x_0)} =
= 12 - \frac{(12)^3 - 72 \times 12 - 220}{3 \times (12)^2 - 72} = 12 - \frac{644}{390} = \frac{919}{90} \approx 10.211$
$\cdots$
$x_3 = x_2 - \frac{f(x_2)}{f'(x_2)} \approx 9.727$
$x_4 \approx 9.727$

**Задание 3.7**

Найдите положительный корень для уравнения $x^2 + 9x - 5 = 0$.

В качестве стартовой точки возьмите $x_0 = 2.2$.

Расчёт произведите поэтапно или с помощью Python.

Ответ округлите до двух знаков после точки-разделителя.

In [17]:
def func1(x):
    return x**2+9*x-5

def func2(x):
    return 2*x+9

init_value = 2.2
iter_count = 0
x_curr = init_value
epsilon = 0.000001
f = func1(x_curr)

while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
print(x_curr)
print(iter_count)

0.5249378105604451
5


$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)} =$  
$= x_n - \frac{x^2_n + 9x_n - 5}{2x_n + 9} = \frac{x^2_n + 5}{2x_n + 9}$

**Задание 3.9**

С помощью метода Ньютона найдите точку минимума для функции $f(x) = 8x^3 - 2x^2 - 450$.

Для расчётов используйте Python.

В качестве стартовой точки возьмите 42, точность примите за 0.0001.

Ответ округлите до трёх знаков после точки-разделителя.

In [18]:
def func(x):
    return 8*x**3-2*x**2-450
def func1(x):
    return 24*x**2 - 4*x
def func2(x):
    return 48*x -4
init_value = 42
iter_count = 0
x_curr = init_value
epsilon = 0.0001
f = func1(x_curr)
while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
print(x_curr)
print(round(x_curr, 3))
print(round(func(x_curr),3))

0.16666666666666785
0.167
-450.019


_________________________________________

### 4. Квазиньютоновские методы

✍ В предыдущем юните мы рассмотрели метод Ньютона. В отличие от градиентного спуска, метод Ньютона использует на каждой итерации не только градиент, но и матрицу Гессе. Это обеспечивает более быструю сходимость к минимуму, но в то же время это приводит к слишком большим вычислительным затратам. В данном юните мы рассмотрим класс методов, в которых решается  эта проблема, — **класс квазиньютоновских методов**.

Напомним, что в методе Ньютона мы обновляем точку на каждой итерации в соответствии со следующим правилом:

$\mathbf{x}_{k+1}=\mathbf{x}_{k}-\left[H\left(\mathbf{x}_{k}\right)\right]^{-1} \nabla f\left(\mathbf{x}_{k}\right)$

На каждом шаге здесь вычисляется гессиан, а также его обратная матрица.

В квазиньютоновских методах вместо вычисления гессиана мы просто аппроксимируем его матрицей, которая обновляется от итерации к итерации с использованием информации, вычисленной на предыдущих шагах. Так как вместо вычисления большого количества новых величин мы использует найденные ранее значения, квазиньютоновский алгоритм тратит гораздо меньше времени и вычислительных ресурсов.

Формально это описывается следующим образом:

$x_{k+1} = x_k - H_k \nabla f(x_k)$

В данном случае вместо обратного гессиана появляется матрица $H_k$, которая строится таким образом, чтобы максимально точно аппроксимировать настоящий обратный гессиан.

Математически это записывается так:

$H_k - \left [\nabla^2 f(x_k) \right ]^{-1} \to 0 \ при \ k \to \infty$

Здесь имеется в виду, что разница между матрицей в квазиньютоновском методе и обратным гессианом стремится к нулю.

Эта матрица обновляется на каждом шаге, и для этого существуют разные способы. Для каждого из способов есть своя модификация квазиньютоновского метода. Эти способы объединены **ограничением**: процесс обновления матрицы должен быть достаточно эффективным и не должен требовать вычислений гессиана. То есть, по сути, на каждом шаге мы должны получать информацию о гессиане, не находя непосредственно сам гессиан.

Если вас интересует математическая сторона обновления и аппроксимации матрицы, прочитайте эту статью. В силу того, что понимание этой части метода требует очень серьёзной математической подготовки, мы опустим её. Однако можем заверить вас, что для успешного использования алгоритма и его понимания знание всех математических выводов не требуется.

**Три самые популярные схемы аппроксимации:**

- симметричная коррекция ранга 1 (SR1);
- схема Дэвидона — Флетчера — Пауэлла (DFP);
- схема Бройдена — Флетчера — Гольдфарба — Шанно (BFGS).

Последняя схема (**BFGS**) самая известная, стабильная и считается наиболее эффективной. На ней мы и остановимся. Своё название она получила из первых букв фамилий создателей и исследователей данной схемы: Чарли Джорджа Бройдена, Роджера Флетчера, Дональда Гольдфарба и Дэвида Шанно.

У этой схемы есть две известных вариации:

- L-BFGS;
- L-BFGS-B.

Обе этих вариации необходимы в случае большого количества переменных для экономии памяти (так как во время их реализации хранится ограниченное количество информации). По сути, они работают одинаково, и L-BFGS-B является лишь улучшенной версией L-BFGS для работы с ограничениями.

Метод BFGS очень устойчив и на данный момент считается одним из наиболее эффективных. Поэтому, если, например, применить функцию optimize без указания метода в библиотеке SciPy, то по умолчанию будет использоваться именно BFGS либо одна из его модификаций, указанных выше. Также данный метод используется в библиотеке sklearn при решении задачи логистической регрессии.

Рассмотрим **алгоритм применения этого метода**. Постарайтесь понять последовательность действий и основной принцип.

1. Реализация алгоритма начинается с того, что мы задаём начальную точку $x_0$, выбираем точность алгоритма, а также изначальную аппроксимацию для обратного гессиана функции. 

Здесь как раз таится главная проблема: нет никакого универсального рецепта для выбора этого приближения. Можно поступить по-разному: использовать гессиан, вычисленный в изначальной точке, взять единичную матрицу (такая настройка стоит по умолчанию в некоторых функциях библиотек Python) или другую матрицу, если она невырождена и хорошо обусловлена.

2. Когда мы определили, откуда будем начинать, необходимо понять, как попасть в следующую точку. Поэтому на втором шаге мы вычисляем направление для поиска следующей точки:

$p_k = -H_k * \nabla f_k$

3. Далее находим следующую точку, используя соотношение:

$x_{k+1} = x_k + k * p_k$

Здесь важным вопросом является нахождение коэффициента $k$, который регулирует шаг. Его подбирают линейным поиском в соответствии с условиями, о которых можно подробно прочитать здесь.

Нам важно лишь понимать суть: мы находим такое значение k, при котором получим минимальное значение функции $f(x_k + k * p_k)$ (так как мы хотим попасть в минимум). Также важно отметить, что в расчёте коэффициента $k$ участвуют две константы $0 \leq c_1 \leq c_2 \leq 1$. Обычно в качестве их значений берут 0.001 и 0.9 (это считается хорошей эвристикой, показывающей высокие результаты).

4. На следующем шаге необходимо определить два следующих вектора:

$s_{k}=x_{k+1}-x_{k}$  
$y_{k}=\nabla f_{k+1}-\nabla f_{k}$  

Здесь $s_k$ — шаг алгоритма, a $y_k$ — изменение градиента на данной итерации. Они не требовались для перехода в следующую точку и нужны строго для того, чтобы найти следующее приближение обратной матрицы гессиана.

5. После того как мы их нашли, обновляем гессиан, руководствуясь следующей формулой:

$H_{k+1}=\left(I-k * s_{k} * y_{k}^{T}\right) H_{k}\left(I-k * y_{k} * s_{k}^{T}\right)+* s_{k} * s_{k}^{T}$

Не стоит её пугаться. Как уже было сказано выше, эта формула —  результат очень серьёзных и длительных математических исследований. Её не требуется знать наизусть или уметь реализовывать вручную. Однако для полноты повествования мы не можем не привести её.

В данной формуле за I обозначена единичная матрица, $s_k$ и $y_k$ мы вычислили на предыдущем шаге, а $k$ вычисляется следующим образом:

$k=\frac{1}{y_{k}^{T} s_{k}}$

Алгоритм довольно сложный, поэтому давайте рассмотрим **пример** ↓

Сразу оговоримся, что несколько шагов в этом алгоритме мы приведём без ручных расчётов (например, нахождение следующей аппроксимации гессиана) в силу их высокой сложности. Постарайтесь сильнее всего сконцентрироваться на шагах решения и понять логику работы алгоритма и последовательность действий. Мы начнём реализовывать алгоритм «вручную», а затем вы сможете самостоятельно завершить решение задачи с использованием Python (разумеется, далее будет указано, как это сделать). К сожалению, серьёзные и эффективные методы настолько сложны, что решать с их помощью задачи, используя только лист бумаги, ручку и калькулятор (как мы могли это делать, например, с методом Лагранжа), уже невозможно.

Будем искать экстремум для функции следующего вида:

$f(x, y)=x^{2}-x y+y^{2}+9 x-6 y+20$

В качестве начальной точки выберем следующую:

$x_0 = (1,1)$

Находим градиент для нашей функции:

$\begin{aligned} f_{x}^{\prime} &=\left(x^{2}-x y+y^{2}+9 x-6 y+20\right)^{\prime}_{ x}=\\ &=2 x-y+0+9-0+0=2 x-y+9 \\ f^{\prime}_{ y} &=\left(x^{2}-x y+y^{2}+9 x-6 y+20\right)^{\prime} _{y}=\\ &=0-x+2 y+0-6+0=-x+2 y-6 \end{aligned}
\nabla f=\left(\begin{array}{c}2 x-y+9 \\ -x+2 y-6\end{array}\right)$

Начинаем первую итерацию с точки x_0 = (1,1). Вычисляем для неё градиент:

$\nabla f=\left(\begin{array}{c}2 x-y+9 \\ -x+2 y-6\end{array}\right) \quad=\left(\begin{array}{c}2 \cdot 1-1+9 \\ -1 \times 2 \cdot 1-6\end{array}\right)=\left(\begin{array}{c}10 \\ -5\end{array}\right)$

Теперь необходимо выяснить, стоит ли заканчивать поиск (ведь, возможно, мы уже в минимуме). Для этого находим длину вектора градиента:

$\left|\nabla f(x_0) \right| = \sqrt{10^2 + (-5)^2} = 11.18$

Сравниваем полученный результат с точностью, которая нам необходима. Допустим, мы хотим достигнуть точности 0.001:

$\left|\nabla f(x_0) \right| = 11.18 > 0.001$

Итак, точность не достигнута, так как градиент в экстремуме должен быть равен или очень близок к нулю. Это значит, что надо искать дальше.

Теперь необходимо определить, в каком направлении искать нужную точку. Для этого выполняем следующие вычисления, согласно нашему алгоритму:

$p_0 = -H_0 * \nabla f(x_0) = - \begin{pmatrix} 1 & 0 \\ 0 & 1 \\ \end{pmatrix} \begin{pmatrix} 10 \\ -5 \end{pmatrix} = \begin{pmatrix} -10 \\ 5 \end{pmatrix}$  
$x_0 + \alpha_0 * p_0 = (1,1) + \alpha_0 (10,-5) = (1 - 10 \alpha_0, 1 + 5 \alpha_0)$  
$f(\alpha_0) = (1 - 10 \alpha_0)^2 - (1 - 10 \alpha_0) (1 + 5 \alpha_0) + (1 + 5 \alpha_0)^2 + 9 (1 - 10 \alpha_0) - 6 (1 + 5 \alpha_0) + 20$

Упрощаем выражение:

$1-20 a_{0}+100 a_{0}^{2}-1+10 a_{0}-5 a_{0}+50 a_{0}^{2}+1+10 a_{0}+25 a_{0}^{2}+9-90 a_{0}-6-30 a_{0}+20= 175 a_{0}^{2}-125 a_{0}+24$

Находим производную от результата:

$\frac{\partial f}{\partial a_0}=350a_{0}-125=0 \Rightarrow a_{0}=0.357$

Теперь мы можем найти следующую точку:

$x_{1}=x_{0}+{a }_{0} * p_{0}=(-2.571,2.786)$  
$s_{0}=x_{1}-x_{0}=(-2.571,2.786)-(1,1)=(-3.571,1.786)$

Вычисляем значение градиента в следующей найденной точке, т. е. в $x_1$:

$\nabla f\left(x_{1}\right)=\left(\begin{array}{l}1.071 \\ 2.143\end{array}\right)$  
$y_{0}=\nabla f\left(x_{1}\right)-\nabla f\left(x_{0}\right)=(1.071,2.143)-(10,-5)=(-8.929,7.143)$  

Находим приближение гессиана:

$H_{1}=\left(\begin{array}{ll}0.694 & 0.367 \\ 0.367 & 0.709\end{array}\right)$

Проверяем, стоит ли остановиться в этой точке:

$\left|\nabla f(x_1) \right| = 2.396 > 0.001$

К сожалению, необходимая точность всё ещё не достигнута.

Перечисленные выше шаги стоит повторять до тех пор, пока не будет достигнуто значение, близкое к нулю (меньшее, чем изначально заданная точность).

Разумеется, при решении прикладных задач не понадобится делать ничего подобного, ведь мы умеем программировать и знаем, что в библиотеках Python есть все необходимые методы.

Давайте рассмотрим, как с помощью функций Python мы сможем применить квазиньютоновские методы для оптимизации функции $f(x,y) = x^2 + y^2$.

Подгрузим необходимые библиотеки:

In [19]:
import numpy as np
from scipy.optimize import minimize

Определим функцию, которую будем оптимизировать. Вместо отдельных x и y можно взять координаты единого вектора:

In [20]:
def func(x):
    return x[0]**2.0 + x[1]**2.0

Теперь определим градиент для функции:

In [21]:
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])

Зададим начальную точку:

In [22]:
x_0 = [1.0, 1.0]

Определим алгоритм:

In [23]:
result = minimize(func, x_0, method='BFGS', jac=grad_func)

Выведем результаты:

In [24]:
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))
 
#Статус оптимизации Optimization terminated successfully.
#Количество оценок: 3
#Решение: f([0. 0.]) = 0.00000

Статус оптимизации Optimization terminated successfully.
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


Итак, мы получили, что минимум функции достигается в точке (0,0). Значение функции в этой точке также равно нулю.

Можно повторить то же самое с вариацией  L-BFGS-B:

In [25]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 + x[1]**2.0
 
#  определяем градиент функции
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])
 
# определяем начальную точку
x_0 = [1, 1]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


→ Иногда количество итераций у двух модификаций различается, но ответ совпадает. Бывает также, что одна из вариаций может не сойтись, а другая — достичь экстремума, поэтому советуем не воспринимать их как взаимозаменяемые алгоритмы. На практике лучше пробовать разные варианты: если у вас не сошёлся алгоритм BFGS, можно попробовать L-BFGS-B, и наоборот. Также можно экспериментировать одновременно с обоими алгоритмами, чтобы выбрать тот, который будет сходиться для функции за меньшее число итераций и тем самым экономить время.

→ Важно понимать, что для некоторых функций не из всех стартовых точек получается достичь сходимости метода. Тогда их можно перебирать, к примеру, с помощью цикла.

✍ Итак, мы обсудили один из самых эффективных на сегодняшний день алгоритмов — вариацию BFGS квазиньютоновских методов. Вы будете регулярно сталкиваться с этим алгоритмом при решении различных задач и при использовании библиотек для оптимизации. Так что давайте попрактикуемся: в этом юните мы посмотрели фрагмент поэтапного разбора метода BFGS для функции $f(x,y) = x^2 - xy + y^2 + 9x - 6y + 20$ — давайте завершим начатое и найдём точку минимума ↓

**Задание 4.1**

Найдите точку минимума для функции $f(x,y) = x^2 - xy + y^2 + 9x - 6y + 20$.

В качестве стартовой возьмите точку (-400, -400).

Значения координат округлите до целого числа.

In [26]:
def func(x):
    return x[0] ** 4.0 - x[0] * x[1] + x[1] ** 2 + 9 * x[0] - 6 * x[1] + 20
def grad_func(x):
    return np.array([2 * x[0] - x[1] + 9, -x[0] + 2 * x[1] - 6])

x_0 = [-400, -400]
result = minimize(func, x_0, method='BFGS', jac=grad_func)
solution = result['x']
print (solution)

[-3.99999852  1.00000155]


**Задание 4.4**

Найдите минимум функции $f(x) = x^2 - 3x + 45$ с помощью квазиньютоновского метода BFGS.

В качестве стартовой точки возьмите $x = 10$.

В качестве ответа введите минимальное значение функции в достигнутой точке.


In [27]:
from scipy.optimize import minimize
def func(x):
    return x[0]**2.0 - 3*x[0] + 45
def grad_func(x):
    return 2*x[0]-3
x_0 = 10
result = minimize(func, x_0, method='BFGS', jac=grad_func)
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации Optimization terminated successfully.
Количество оценок: 5
Решение: f([1.5]) = 42.75000


**Задание 4.5**

Решите предыдущую задачу, применяя модификацию L-BFGS-B.

В каком случае получилось меньше итераций?

In [28]:
x_0 = 10
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3
Решение: f([1.5]) = 42.75000


**Задание 4.7**

Найдите минимум функции $f(x,y) = x^4 + 6*y^2 + 10$, взяв за стартовую точку (100,100).

Какой алгоритм сошелся быстрее?

In [29]:
def func(x):
    return x[0]**4.0 + 6*x[1]**2.0 + 10
def grad_func(x):
    return np.array([4* x[0] ** 3, 12* x[1]])
x_0 = [100.0, 100.0]
result = minimize(func, x_0, method='BFGS', jac=grad_func)
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации Optimization terminated successfully.
Количество оценок: 37
Решение: f([1.31617159e-02 6.65344582e-14]) = 10.00000


In [30]:
def func(x):
    return x[0]**4.0 + 6*x[1]**2.0 + 10
def grad_func(x):
    return np.array([4* x[0] ** 3, 12* x[1]])
x_0 = [100.0, 100.0]
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Количество оценок: 40
Решение: f([-9.52718297e-03 -2.32170510e-06]) = 10.00000


_____________________________________________________

### 5. Линейное программирование

✍ В предыдущем модуле мы уже немного говорили о задачах условной оптимизации. Теперь пришло время вернуться к ним и рассмотреть их более основательно. Мы разберём один из наиболее часто встречающихся случаев задачи условной оптимизации — **задачу линейного программирования**.

**Линейное программирование** — это метод оптимизации для системы линейных ограничений и линейной целевой функции. Целевая функция определяет оптимизируемую величину, и цель линейного программирования состоит в том, чтобы найти значения переменных, которые максимизируют или минимизируют целевую функцию.

Линейное программирование полезно применять для многих задач, требующих оптимизации ресурсов:

- В производстве — чтобы рассчитать человеческие и технические ресурсы и минимизировать стоимость итоговой продукции.
- При составлении бизнес-планов — чтобы решить, какие продукты продавать и в каком количестве, чтобы максимизировать прибыль.
- В логистике — чтобы определить, как использовать транспортные ресурсы для выполнения заказов за минимальное время.
- В сфере общепита — чтобы составить расписание для официантов.

Задача линейного программирования — это задача оптимизации, в которой целевая функция и функции-ограничения линейны, а все переменные неотрицательны.

Есть и разновидность задачи линейного программирования, в которой мы знаем, что все переменные могут принимать только целочисленные значения. Например, если переменные в нашей задаче — это количество людей или произведённых на заводе изделий, то они, разумеется, не могут быть дробными.

**Целочисленным линейным программированием (ЦЛП)** называется вариация задачи линейного программирования, когда все переменные — целые числа.

Давайте рассмотрим алгоритм решения задачи линейного программирования на конкретном **примере** ↓

**? Пример № 1**

Фабрика игрушек производит игрушки-антистресс и игрушки-вертушки.

Для изготовления игрушки-антистресс необходимо потратить 2 доллара  и 3 часа, для изготовления игрушки-вертушки — 4 доллара и 2 часа.

На этой неделе в бюджете у фабрики есть 220 долларов, и оплачено 150 трудочасов для производства указанных игрушек.

Если одну игрушку-антистресс можно продать за 6 долларов, а игрушку-вертушку — за 7, то сколько экземпляров каждого товара необходимо произвести на этой неделе, чтобы максимизировать прибыль?

Такая задача идеально подходит для использования методов линейного программирования по следующим причинам:

- все условия являются линейными;
- значения переменных каким-то образом ограничены;
- цель состоит в том, чтобы найти значения переменных, которые максимизируют некоторую величину.

Обратите внимание, что производство каждой детали связано с затратами, как временными, так и финансовыми. Изготовление каждой игрушки-антистресс стоит 2 доллара, а изготовление каждой вертушки — 4 доллара. У фабрики есть только 220 долларов, чтобы потратить их на производство изделий. Отсюда возникает ограничение на возможное количество изготовленных товаров.

Обозначим за $x$ количество произведённых игрушек-антистресс, за $y$ — количество произведённых игрушек-вертушек Тогда это ограничение можно записать в виде следующего неравенства:

$2 x+4 y \leq 220$

Также существует ограничение на то, сколько времени мы можем потратить на производство игрушек. На изготовление каждой игрушки-антистресс уходит 3 часа, а на изготовление каждой игрушки-вертушки — 2 часа. На этой неделе у фабрики есть только 150 рабочих часов, так что производство ограничено по времени. Это ограничение можно записать в виде неравенства:

$3 x+2 y \leq 150$

К этим ограничениям мы также можем добавить ограничения из соображений здравого смысла. Невозможно произвести отрицательное количество игрушек, поэтому необходимо обозначить следующие ограничения:

$\begin{aligned} &x \geq 0 \\ &y \geq 0 \end{aligned}$

Итак, мы записали все необходимые ограничения. Они образуют систему неравенств:

$\left\{\begin{aligned} 2 x+4 y & \leq 220 \\ 3 x+2 y & \leq 150 \\ x & \geq 0 \\ y & \geq 0  \end{aligned}\right.$

Если представить систему этих неравенств в графическом виде, получим многоугольник:

<img src="data\MATHML_md6_5_1.png" alt="drawing" width="400"/>

Закрашенная область является областью допустимых решений этой задачи. Наша цель — найти внутри этого многоугольника такую точку, которая даст наилучшее решение нашей задачи (максимизацию или минимизацию целевой функции).

Итак, мы поняли, что нам нужно найти максимальное значение прибыли (целевой функции) для точек внутри области допустимых значений, однако самой целевой функции у нас пока нет. Давайте составим её. На изготовление каждой игрушки-антистресс требуется 2 доллара, а продать её можно за 6. Получается, что чистая прибыль от продажи составляет 4 доллара. Чтобы изготовить игрушку-вертушку, мы потратим 4 доллара, а продадим её за 7. Значит, чистая прибыль для вертушки составляет 3 доллара. Исходя из этого, получаем целевую функцию для суммарной прибыли:

$p(x, y)=4 x+3 y$

Нам необходимо найти максимально возможное значение этой функции с учётом того, что точка, в которой оно будет достигаться, должна удовлетворять условиям системы, которую мы написали ранее.

Для того чтобы решить задачу, выразим одну переменную через другую (так удобнее строить графики линейной функции в стандартной системе координат):

$y=-\frac{4}{3} x+\frac{P}{3}$

На графике ниже наша линия изображена красным цветом. Она может двигаться вверх и вниз в зависимости от значения P (вы можете видеть три прямых — это три разных положения одной и той же прямой).

Попробуем найти такую точку, для которой значение y будет наибольшим. Нас это интересует, так как мы хотим максимизировать значение $P$, а при максимально возможном $P$ прямая поднимется настолько высоко, насколько это возможно, и пересечение с точкой ординат тоже будет находиться максимально высоко.

<img src="data\MATHML_md6_5_2.png" alt="drawing" width="400"/>

Линия, которая максимизирует точку пересечения c осью ординат, проходит через точку (20,45) — это точка пересечения первых двух ограничений. Все остальные прямые, которые проходят выше, не проходят через область допустимых решений. Все остальные «нижние» прямые проходят более чем через одну точку в допустимой области и не максимизируют пересечение прямой с осью ординат, так как находятся ниже.

Таким образом, получаем, что завод должен произвести 20 игрушек-антистресс и 45 игрушек-вертушек. Это даст прибыль в размере 215 долларов:

$20*4 + 45* 3 = 215$

**? Пример № 2**

Фермер кормит своих коров специальной смесью. Для её изготовления он использует два вида корма: на 1 кг кукурузного корма приходится 100 г белка и 750 г крахмала, на 1 кг пшеничного — 150 г белка и 700 г крахмала.

Каждой корове необходимо давать не более 7 кг корма в сутки, иначе у неё заболит живот и придётся тратить деньги на ветеринара. При этом, чтобы давать оптимально полезное и вкусное молоко, каждая корова должна ежедневно потреблять как минимум 650 г белка и 4000 г крахмала.

Известно, что кукурузный корм стоит 0.4 доллара за 1 кг, а пшеничный — 0.45 долларов за 1 кг.

Какая кормовая смесь будет минимизировать затраты и в то же время позволит получать качественное молоко?

Обозначим за $c$ количество кукурузного корма в смеси, а за $w$ — количество пшеничного. Тогда ограничения можно выразить следующим образом:

$\left\{\begin{aligned} 0.1 c+0.15 w & \geq 0.65 \\ 0.75 c+0.7 w & \geq 4 \\ c+w & \leq 7 \\ c & \geq 0 \\ w & \geq 0 \end{aligned}\right.$

Минимизировать мы будем целевую функцию, отражающую затраты на корм и имеющую следующий вид:

$f(c, w)=0.40 c+0.45 w$

Снова изобразим условие задачи графически. Начнём с многоугольника области допустимых значений:

<img src="data\MATHML_md6_5_3.png" alt="drawing" width="400"/>

Получается, что нам необходимо найти точку, в которой пересекаются две прямые, обозначающие условия. Можно записать это следующим образом (точка пересечения — решение системы):

$\left\{\begin{array}{l} 0.1 c+0.15 w=0.65 \\ 0.75 c+0.7 w=4 \end{array}\right.$

Найдя эту точку (для этого можно воспользоваться методом Гаусса или программированием), можно найти и итоговую стоимость смеси:

$f(3.411,2.059)=\$ 2.29$

Отлично, мы научились решать задачи линейного программирования. Теперь давайте закрепим полученные знания ↓

**Задание 5.5**

Магазин спортивных товаров получает прибыль в размере 6 долларов с каждого проданного футбольного мяча и 5.5 долларов — с бейсбольного.

Каждый месяц магазин продаёт от 35 до 45 футбольных мячей и от 40 до 55 бейсбольных.

Известно, что в этом месяце у магазина есть в общей сложности 80 мячей.

Какую максимальную прибыль в этом месяце может получить магазин от продажи мячей?

$\left\{\begin{aligned} 35 \leq x & \leq 45 \\ 40 \leq y & \leq 55 \\ x+y &=80 \\ \end{aligned}\right.$

Максимизируем следующую функцию:

$6 \cdot x+5.5 \cdot y$

Получаем, что оптимальная точка: x =40 и y = 40.

$6\cdot 40 + 5.5 \cdot 40 = 460$

**Задание 5.6**

На текстильной фабрике есть 750 метров хлопка и 1000 метров полиэстера.

Для изготовления свитшота требуется 1 метр хлопка и 2 метра полиэстера, для изготовления рубашки — 1.5 метра хлопка и 1 метр полиэстера.

Свитшот можно продать за 30 евро, а рубашку — за 24 евро.

Какое суммарное количество свитшотов и рубашек максимизирует возможную прибыль?

Обозначим за S количество произведённых свитшотов, а за C — количество произведённых рубашек. Запишем
ограничения:

$\begin{aligned} &S \geq 0 \\ &C \geq 0 \\ &S+1.5 C \leq 750 \\ &2 S+C \leq 1000 \end{aligned}$

Максимизируем следующую функцию:
$V(S, C)=30 S+24 C$

<img src="data\pic-2.png" alt="drawing" width="400"/>


$V(0,0) = 0 \ EUR$  
$V(500,0) = 500 \cdot 30 = 15000 \ EUR$
$V(0,500) = 500 \cdot 24 = 12000 \ EUR$  
$V(375,250) = 375 \cdot 30 + 250 \cdot 24 = 17250 \ EUR$  
$375 + 250 = 625$

В следующем юните мы научимся решать задачи линейного программирования с использованием библиотек Python. Однако для того чтобы грамотно передавать условия в функции, необходимо научиться формулировать задачи линейного программирования в матричном виде. Давайте рассмотрим несколько примеров в качестве подготовки ↓

**? Пример № 1**

Вы отвечаете за рекламу в компании.

Затраты на рекламу в месяц не должны превышать 10 000 руб. Один показ рекламы в интернете стоит 1 рубль, а на телевидении — 90 рублей. При этом на телевидении нельзя купить больше 20 показов.

Практика показывает, что 1 показ телерекламы приводит в среднем 300 клиентов, а 1 показ в интернете — 0.5 клиента.

Вам необходимо привести как можно больше клиентов.

Обозначим за $x_1$ количество показов в интернете, за $x_2$ — количество показов на телевидении. Будем максимизировать число приведённых клиентов $0.5 x_1 + 300 x_2$.

Составим задачу минимизации с ограничением на количество показов и затраты. Наша целевая функция примет следующий вид:

$\min \left(-0.5 x_{1}-300 x_{2}\right)$

Мы заменили максимизацию на минимизацию, так как готовим задачу для решения стандартным алгоритмом, а по умолчанию задача оптимизации сводится к минимизации. Поэтому перед применением функций Python формулируйте задачу именно в формате поиска минимума.

Ограничения можно выразить так:

$\left\{\begin{array}{l}x_{2} \leq 20 \\ x_{1}+90 x_{2} \leq 10000\end{array}\right.$

Разумеется, будем помнить, что количество показов рекламы всегда неотрицательное.

Представим наши данные в векторном виде. Пусть $c$ — это вектор приведённых клиентов:

$c=\left(\begin{array}{c}-0.5 \\ -300\end{array}\right)$

За $A$ и $b$ мы возьмём такие матрицы, чтобы с их помощью можно было представить систему ограничений:

$A=\left(\begin{array}{cc}0 & 1 \\ 1 & 90\end{array}\right), \ b=\left(\begin{array}{c}20 \\ 10000\end{array}\right)$

Тогда наша задача формулируется следующим образом:

$\operatorname{min} c^{T} x$  
$A x \leq b$

Итак, у нас получилось перевести задачу на язык линейной алгебры. Решать такие задачи мы будем в следующем юните, а пока рассмотрим ещё один пример посложнее ↓

**? Пример № 2**

Есть $n$ задач и $n$ человек, которые могут их выполнить.

Каждая задача должна быть сделана одним человеком, и каждый должен сделать ровно одну задачу.

Выполнение задачи $j$ человеком $i$ будет стоить $c_{ij}$.

Вам необходимо сделать все задачи как можно дешевле.

Так как очевидно, что люди и задачи могут быть выражены только целыми числами, будем формулировать задачу как задачу целочисленного программирования.

Пусть $x_{ij} = 1$, если задачу $j$ выполнил человек $i$. Если работы выполнил кто-то другой, то $x_{ij} = 0$.

Распределение работ по людям мы можем представить в виде таблицы-матрицы, например, следующим образом:

<img src="data\MATHML_md6_5_7_new.png" alt="drawing" width="400"/>

Здесь на пересечениях столбцов-работ и строк-людей указана стоимость работы, если её выполнит выбранный человек. Например, второй человек выполнит первую работу за 1 денежную единицу, вторую — за 5 денежных единиц, а третью — за 2 денежных единицы.

Если мы выберем первого человека для выполнения первой работы, второго — для третьей и третьего — для второй (как и закрашено на рисунке выше), тогда в матрице распределения работ мы обозначим соответствующие элементы за 1, а остальные — за 0.

Теперь сформулируем задачу. Минимизируем суммарную стоимость $c_{ij} \cdot x_{ij}$:

$\min \sum_{i, j} c_{i j} x_{i j}$

Теперь запишем условия:

$x$ равен либо 0, либо 1: $x_{i j} \leq 1$.  
Каждый человек должен взять ровно одну задачу: $\forall i: \sum_{j} x_{i j}=1$.  
Каждую задачу должен взять ровно один человек: $\forall j: \sum_{i} x_{i j}=1$.  
Мы сформулировали задачи линейного программирования в чётком математическом виде. Самое время переходить к следующему юниту и учиться решать задачи с использованием программирования →

**ДОПОЛНИТЕЛЬНО**

Если вам интересно изучить более сложные, но очень известные задачи оптимизации, к которым применимо линейное программирование, рекомендуем прочитать следующие статьи:

- Задача коммивояжёра https://habr.com/ru/post/560468/  
- Транспортная задача https://habr.com/ru/post/573224/  

_________________________

### 6. Практика: линейное программирование

✍ Мы познакомились с задачами линейного программирования, и теперь пришло время научиться решать их с использованием функций Python.

В языке Python есть множество библиотек, с помощью которых можно решить задачу линейного программирования. Вот основные, которые мы рассмотрим в данном юните:

- **SciPy** (scipy.optimize.linprog);  
- **CVXPY**;   
- **PuLP**.  

У каждой библиотеки есть свои особенности использования, и большинство задач можно решить с помощью любой из них. Давайте посмотрим все варианты, чтобы у вас всегда был выбор — решим по одной задаче для каждой библиотеки.

**? Пример № 1. SciPy** (scipy.optimize.linprog)

У нас есть 6 товаров с заданными ценами на них и заданной массой.

Вместимость сумки, в которую мы можем положить товары, заранее известна и равна 15 кг.

Какой товар и в каком объёме необходимо взять, чтобы сумма всех цен товаров была максимальной?

Создадим переменные на основе предложенных данных:

In [2]:
import numpy as np

values = [4, 2, 1, 7, 3, 6] #стоимости товаров
weights = [5, 9, 8, 2, 6, 5] #вес товаров
C = 15 #вместимость сумки
n = 6 #количество товаров

Сформулируем задачу линейного программирования. Максимизируем произведение стоимости на количество, учитывая, что произведение веса на искомое количество товаров должно укладываться во вместимость сумки:

$\max \sum v_{i} x_{i}$  
$\sum w_{i} x_{i} \leq C$  

Из предыдущего юнита мы уже знаем, что в векторно-матричной форме наша задача должна формулироваться в следующем виде:

$\operatorname{min} c^{T} x$  
$A x \leq b$  

Получается, что в наших обозначениях мы имеем следующее:

$c=-v, \ A=w^{T}, \ b=(C)$

Здесь нам необходимо вспомнить линейную алгебру, так как очень важно, чтобы векторы были в нужных нам размерностях, иначе мы не сможем использовать матричное умножение. Вектор $A$ размера 6 мы превращаем в матрицу размера (1, 6) с помощью функции expand_dims(). Создаём все необходимые переменные:


In [3]:
c = - np.array(values) #изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  #конвертируем список с весами в массив
A = np.expand_dims(A, 0) #преобразуем размерность массива
b = np.array([C]) #конвертируем вместимость в массив

Передаём подготовленные переменные в оптимизатор SciPy:

In [4]:
from scipy.optimize import linprog
linprog(c=c, A_ub=A, b_ub=b)

     con: array([], dtype=float64)
     fun: -52.50000000003075
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([-2.24904539e-11])
  status: 0
 success: True
       x: array([6.18738537e-14, 1.05853307e-12, 1.21475944e-13, 7.50000000e+00,
       4.00246695e-13, 4.71394166e-13])

Получаем искомое значение функции — 52.5 (в выводе значение с минусом, но мы меняем знак, возвращаясь к задаче максимизации). x = (0, 0, 0, 7.5, 0, 0). Таким образом, мы взяли только самую дорогую, четвёртую вещь. Она одна весит 2 кг, а если взять её 7.5 раз, то получится как раз 15 кг. Отлично, задача решена.

**? Пример № 2. CVXPY**

Снова решим задачу из примера № 1, но уже предположим, что товары нельзя дробить, и будем решать задачу целочисленного линейного программирования.

SciPy не умеет решать такие задачи, поэтому будем использовать новую библиотеку CVXPY.

Важно! С установкой этот библиотеки порой возникают проблемы. Если вы столкнулись с трудностями, посоветуйтесь с ментором или воспользуйтесь Google Colaboratory.

С помощью CVXPY создадим переменную-массив. Укажем его размерность, а также условие, что все числа в массиве должны быть целыми:

In [42]:
!pip install cvxpy
!pip install pyscipopt
!pip install cvxopt
!pip install glpk

"pip" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.
"pip" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.
"pip" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.
"pip" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [5]:
from sympy.solvers import solve
import cvxpy
#from cvxopt import glpk
x = cvxpy.Variable(shape=n, integer = True)

Далее зададим ограничения, используя матричное умножение:

In [6]:
A = A.flatten() # Преобразуем размерность массива
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
total_value = cvxpy.sum(cvxpy.multiply(x, c))

Переходим непосредственно к решению задачи:

In [7]:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint])

Вызываем получившееся решение:

In [8]:
problem.solve()

-inf

В результате получаем бесконечность. Это совершенно нереалистично.

В таком случае будем рассматривать только положительные значения $x$:

$x \geq 0$

В переформулированном виде задача будет решаться следующим образом:

In [9]:
x = cvxpy.Variable(shape=n, integer=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

problem.solve()
x.value

array([0., 0., 0., 7., 0., 0.])

Здесь мы уже получаем 49, и берём только четвёртый товар в количестве семи штук. Можно увидеть, что результат, в целом, очень близок к первому, когда мы использовали библиотеку SciPy — различие лишь в добавлении целочисленности. Значит, у нас получилось решить задачу, когда мы добавили недостающее условие.

А что если мы можем брать не любое количество товаров, а только один или не брать их вовсе? Задаём x типа boolean.

$x= 0 \ или \ x = 1$

Программное решение такой задачи имеет следующий вид:

In [10]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

problem.solve()
x.value

array([1., 0., 0., 1., 0., 1.])

Получим стоимость, равную 17, взяв первый, четвёртый и шестой товары.

Обратите внимание, что, используя SciPy, мы могли не указывать явно, что x только положительные, так как в линейном программировании считаются только неотрицательные x.

А вот CVXPY универсальна. Мы просто задали функцию, не указывая, что это линейное программирование. CVXPY «поняла», что это задача оптимизации, и использовала нужные алгоритмы. Поэтому здесь ограничение на положительные x мы указывали явно.

**? Пример № 3. PuLP**

В нашей каршеринговой компании две модели автомобилей: модель A и модель B. Автомобиль A даёт прибыль в размере 20 тысяч в месяц, а автомобиль B — 45 тысяч в месяц. Мы хотим заказать на заводе новые автомобили и максимизировать прибыль. Однако на производство и ввод в эксплуатацию автомобилей понадобится время:

Проектировщику требуется 4 дня, чтобы подготовить документы для производства каждого автомобиля типа A, и 5 дней — для каждого автомобиля типа B.
Заводу требуется 3 дня, чтобы изготовить модель A, и 6 дней, чтобы изготовить модель B.
Менеджеру требуется 2 дня, чтобы ввести в эксплуатацию в компании автомобиль A, и 7 дней —  автомобиль B.
Каждый специалист может работать суммарно 30 дней.

In [49]:
pip install pulp

     ---------------------------------------- 14.3/14.3 MB 1.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import pulp
from pulp import *

Целевая функция будет выглядеть следующим образом:

$20000 A + 45000 B$

Также запишем ограничения:

$A, \ B \geq 0$  
$4A + 5B \leq 30$   
$3A +6B \leq 30$  
$2A + 7B \leq 30$

Заметьте, что здесь мы снова пишем обычные неравенства, а не условия в матричном виде. Дело в том, что для данной библиотеки так «удобнее», так как она принимает все условия в «первичном» виде.

In [13]:
problem = LpProblem('Производство машин', LpMaximize)
A = LpVariable('Автомобиль A', lowBound=0 , cat=LpInteger)
B = LpVariable('Автомобиль B', lowBound=0 , cat=LpInteger)
#Целевая функция
problem += 20000*A + 45000*B 
#Ограничения
problem += 4*A + 5*B <= 30 
problem += 3*A + 6*B <=30
problem += 2*A + 7*B <=30
problem.solve()
print("Количество автомобилей модели А: ", A.varValue)
print("Количество автомобилей модели В: ", B.varValue)
print("Суммарный доход: ", value(problem.objective))
#Количество автомобилей модели А:  1.0
#Количество автомобилей модели В:  4.0
#Суммарный доход:  200000.0

Количество автомобилей модели А:  1.0
Количество автомобилей модели В:  4.0
Суммарный доход:  200000.0


Выходит, что необходимо произвести 1 автомобиль типа A и 4 автомобиля типа B. Тогда суммарный чистый доход будет равен 200 тысячам.

**Задание 6.1**

Составьте оптимальный план перевозок со склада № 1 и склада № 2 в три торговых центра с учётом тарифов, запасов на складах и потребностей торговых центров, которые указаны в таблице:

<img src="data\MATHML_md6_6_1_1.png" alt="drawing" width="500"/>

Сформулируйте предложенную задачу как задачу линейного программирования и решите её любым способом (желательно программным).

В качестве ответа введите минимальную суммарную стоимость поставки. Ответ округлите до целого числа..

In [14]:
from scipy.optimize import linprog
import numpy as np

cost = np.array([
    [2, 5, 3],
    [7, 7, 6]
])
stock = np.array([180, 220])
demand = np.array([110, 150, 140])
num_warehouse = 2
num_clients = 3

$x_{ij}$ - сколько забирается со i склада клиенту j  
$$f = \sum_{i,j} cost_{ij} * x_{ij}$$

In [15]:
c = cost.flatten()
print(c) 

[2 5 3 7 7 6]


Для каждого склада количество взятых предметов должно быть меньше, чем на складе:

$$\forall i: \sum_j x_{ij} \leq stock_i$$

In [16]:
A = []
b = []
for i in range(0, num_warehouse):
    A.append([0] * (num_clients * i) + [1] * num_clients + [0] * (num_clients * (num_warehouse - i - 1)))
    b.append(stock[i])
A = np.asarray(A)
b = np.asarray(b)
print(A)
print(b)

[[1 1 1 0 0 0]
 [0 0 0 1 1 1]]
[180 220]


Для каждого клиента количество приобретаемых товаров должно быть больше на единицу, чем спрос:

$$\forall j: \sum_i x_{ij} \geq demand_j$$

Который также:

$$\forall j: - \sum_i x_{ij} \leq -demand_j$$

In [17]:
A = A.tolist()
b = b.tolist()
for j in range(0, num_clients):
    A.append(([0] * j + [-1] + [0] * (num_clients - j - 1)) * num_warehouse)
    b.append(-demand[j])
A = np.asarray(A)
b = np.asarray(b)
print(A)
print(b)

[[ 1  1  1  0  0  0]
 [ 0  0  0  1  1  1]
 [-1  0  0 -1  0  0]
 [ 0 -1  0  0 -1  0]
 [ 0  0 -1  0  0 -1]]
[ 180  220 -110 -150 -140]


In [18]:
linprog(c=c, A_ub=A, b_ub=b)

     con: array([], dtype=float64)
     fun: 1899.9999725510097
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([ 2.61579072e-06,  3.20307387e-06, -1.59710822e-06, -2.18403449e-06,
       -2.03772191e-06])
  status: 0
 success: True
       x: array([1.09999998e+02, 1.64001118e-08, 6.99999991e+01, 9.36756152e-08,
       1.49999998e+02, 6.99999989e+01])

Ответ: 110 единиц со склада 1 клиенту 1, 70 единиц со склада 1 клиенту 3,
150 наименований со склада 2 клиенту 2, 70 наименований со склада 2 клиенту 3

**Задание 6.2**

В прошлом юните мы обсуждали задачу о назначениях исполнителей задач - теперь пришло время решить её.

Напомним суть: необходимо распределить пять задач между пятью исполнителями таким образом, чтобы суммарные затраты на работы были наименьшими.

<img src="data\MATHML_md6_6_2.png" alt="drawing" width="300"/>

В качестве ответа введите минимальную стоимость работ.

In [19]:
from scipy.optimize import linprog
import numpy as np
import cvxpy as cvx

#### Матрица стоимостей $C$

In [20]:
c = np.array([[1000, 12, 10, 19, 8],
    [12, 1000, 3, 7, 2], 
    [10, 3, 1000, 6, 20], 
    [19, 7, 6, 1000, 4], 
    [8, 2, 20, 4, 1000]])

#### Матрица переменных $X$

In [21]:
x = cvx.Variable(shape=(5,5), boolean=True)

#### Ограничения (сумма $X$ по строкам и столбцам равна 1)

In [22]:
constraints = [
    cvx.sum(x, axis=0) == np.ones(5),
    cvx.sum(x, axis=1) == np.ones(5)
]
    

#### Целевая функция (сумма $CX$)

In [23]:
func = cvx.sum(cvx.multiply(x, c))
problem = cvx.Problem(cvx.Minimize(func), constraints=constraints)

#### Ответ

In [24]:
problem.solve()

32.0

#### Выбранные ячейки матрицы

In [25]:
np.round(x.value)

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.]])

**Задание 6.3**

Найдите кратчайший маршрут из точки A, который проходит через все другие точки и возвращается в A.

<img src="data\MATHML_md6_6_3.png" alt="drawing" width="300"/>

Сформулируйте эту задачу как задачу ЦЛП и решите её. В качестве ответа укажите длину кратчайшего пути.

In [26]:
import cvxpy as cvx
import numpy as np

Пусть X будет массивом булевых значений 5x5. Если X[A, B] = True, то маршрут проходит по ребру (A, B).

In [27]:
x = cvx.Variable(shape=(5, 5), boolean=True)

Пусть C будет массивом целочисленных значений, в котором хранятся расстояния между вершинами.

In [28]:
c = np.array(
    [
        [0, 12, 10, 19, 8],
        [12, 0, 3, 7, 2],
        [10, 3, 0, 6, 20],
        [19, 7, 6, 0, 4],
        [8, 2, 20, 4, 0],
    ]
)

Зададим ограничения. Нетрудно заметить, что кратчайший маршрут будет соединять ровно 5 пар точек – то есть, сумма x по обеим осям будет равна 5.

In [29]:
constraints = [
    cvx.sum(x, axis=0) == np.ones(5),
    cvx.sum(x, axis=1) == np.ones(5),
]

Также заметим, что "нулевое ребро" между каждой точкой и ней самой же не может входить в наш маршрут – то есть, на главной диагонали X располагаются нули.

In [30]:
for i in range(5):
    constraints.append(x[i, i] == 0)

Определим функцию, считающую длину маршрута.

In [31]:
func = cvx.sum(cvx.multiply(x, c))

Решим задачу.

In [32]:
problem = cvx.Problem(cvx.Minimize(func), constraints=constraints)

### Ответ

In [33]:
problem.solve()

32.0

Длина наикратчайшего пути – 32.

### Найденный путь
Посмотрим на найденный наикратчайший путь. Для этого взглянем на значения матрицы X, при которых достигается минимальная длина пути.

In [34]:
np.round(x.value)

array([[0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

Это означает, что вершина A - первая в маршруте, B - третья, C - вторая, D - четвёртая. Эту же информацию можно достать из матрицы $X$.  
Искомое решение.

![alt text](https://i.ibb.co/qJzt4LW/image.png)

Обратите внимание, что в заданиях 6.2 и 6.3 для записи двух разных задач используется почти одна и та же система уравнений!

______________________________________

### 7. Дополнительные методы оптимизации. Практика

✍ В этом и предыдущем модулях мы рассмотрели большой спектр методов оптимизации, но, разумеется, их существует гораздо больше — мы же разобрали только основные и самые эффективные.

Давайте кратко рассмотрим ещё два метода, которые не включаются в классы с большим количеством алгоритмов (как, например, класс методов, основанных на градиентном спуске, или класс квазиньютоновских методов), но, тем не менее, иногда встречаются в практических заданиях.

### МЕТОД ИМИТАЦИИ ОТЖИГА

В первую очередь познакомимся с **методом имитации отжига** (simulated annealing).

Данный метод можно использовать, когда нужно оптимизировать достаточно сложную и «неудобную» функцию, то есть, такую, для которой не получится применить градиентные или другие оптимизационные методы.

Метод имитации отжига удобен для применения, так как это метод нулевого порядка, а значит у него намного меньше ограничений.

Метод отжига используется также в ускорении обучения нейронных сетей.

Идея для алгоритма имитации отжига взята из реальной жизни. Во время отжига материал (обычно это металл) нагревают до определённой температуры, а затем медленно охлаждают. Когда материал горячий, у него снижается твёрдость и его проще обрабатывать. Когда он остывает, то становится более твёрдым и менее восприимчивым к изменениям.

Наша задача — получить максимально холодный твёрдый материал. Для этого нужна крепкая кристаллическая решётка, в которой все атомы находятся на местах с минимальной энергией. Смысл в том, что, когда металл сильно нагревается, атомы его кристаллической решётки вынужденно покидают свои положения. Когда начинается охлаждение, они стремятся вернуться в состояние, где будет минимальный уровень определённой энергии. То есть нам было нужно, чтобы металл стал холодным и твёрдым, но мы сначала нагрели его (то есть «ухудшили» ситуацию относительно необходимой), чтобы в итоге (после охлаждения) получить намного более крепкую кристаллическую решётку.

Метод имитации отжига относится к **эвристическим методам**. Это алгоритмы, которые основаны на некоторой математической и логической интуиции и возникают без фундаментальных теоретических предпосылок.

Для таких методов обычно нет исследований и доказательств эффективности для различных случаев, однако есть практические наблюдения, демонстрирующие высокую эффективность. Случается и такое, что эвристический метод доказанно ложный с точки зрения математики, но его всё равно используют в отдельных случаях, где он внезапно показывает высокое качество.

Обобщая, можно сказать, что **эвристический подход** — это подход, не имеющий под собой математического обоснования (а возможно, даже имеющий опровержение для каких-то случаев), но при этом являющийся эффективным и полезным.

Метод отжига является именно таким: с одной стороны, он может быстро находить решения экспоненциально сложных задач, а с другой — не всегда сходится к решению, а также может приводить к поиску только локальных минимумов. Однако метод отжига — действенное средство для решения многих сложных оптимизационных задач. Давайте познакомимся с ним ↓

**Идея метода** заключается в том, чтобы иногда позволять значению функции в точке «ухудшаться» (то есть удаляться от минимума) по аналогии с отжигом, где мы «ухудшали» состояние металла (т. е. нагревали его), чтобы в итоге достичь наилучшего результата.

? Зачем это нужно?

Представим, что мы ищем минимум для такой функции:

<img src="data\MATHML_md6_7_1.png" alt="drawing" width="600"/>

Если мы будем минимизировать такую функцию с помощью градиентного спуска, то неизбежно застрянем в локальном минимуме. Метод отжига может позволить функции удалиться от минимума, и тогда мы сможем «перепрыгнуть» препятствие и достичь глобального минимума.

Метод отжига реализуется по следующему алгоритму:

1. Выбор изначальной точки. Обычно начальная точка выбирается случайным образом. Также нужно выбрать начальную и конечную температуру.
2. Основной алгоритм:
    1. Выбираем случайную точку рядом с нашей точкой.
    2. Оцениваем, переходим ли мы в эту новую точку:
        - Если значение функции в новой точке «лучше» значения функции в нашей точке, то переходим.
        - Если значение функции в новой точке «хуже» значения функции в нашей точке, то всё равно с некоторой вероятностью переходим. Чем ниже температура, тем ниже эта вероятность.
    3. Уменьшаем температуру. Если температура стала ниже конечной, алгоритм прекращает работу.

**Математически** и в более строгом виде это можно **записать следующим образом**:

1. Выбираем начальную точку $x_0$ и определяем счётчик итераций $k$. Также определяем начальную температуру $t_0$, конечную температуру $t_{min}$ и функцию температуры $T(k)$, по которой температура будет уменьшаться с количеством итераций.
2. На каждой итерации $k$ выбираем случайную точку $x^*$ из окрестности точки $x_{k}$.
3. Если мы видим, что значение функции в точке $x^*$ меньше значения функции в точке $x_{k}$, то переходим в новую точку:
$f\left(x^{*}\right)<f\left(x_{k}\right)\to x_{k+1}=x^{*}$  
Если же значение функции в точке $x^*$ больше значения функции в точке $x_{k}$, то с вероятностью $e^{\frac{f(x_{k})-f(x^*)}{t_{k}}}$ мы всё равно переходим в новую точку:
$f(x^*)>f(x_k)\;\; \& \;\;p<e^{\frac{f(x_{k})-f(x^*)}{t_{k}}}\;\to\; x_{k+1}=x^{*}$  
4. Понижаем температуру:
$t_{k+1}=T(k)$  
Если температура $t_{k+1}$ меньше конечной температуры $t_{min}$, то алгоритм прекращает работу.
5. Увеличиваем счётчик итераций на 1:
$k=k+1$  
Если мы применяем этот метод, то нам предстоит ответить на два вопроса:

Как выбирать начальную точку $x_0$?
Как выбирать функцию температуры $T(k)$?
Первое значение, как правило, определяется случайно. Второе — в зависимости от того, насколько долго мы хотим, чтобы наш алгоритм работал. **Главное правило**: функция температуры $T(k)$ обязательно должна быть убывающей.

У метода отжига есть **ряд плюсов и минусов**.

<img src="data\happy-icon.png" alt="drawing" width="50"/>  

- Не использует градиент. Это значит, что его можно использовать для функций, которые не являются непрерывно дифференцируемыми.  
- Можно использовать даже для дискретных функций. Причём именно для дискретных функций метод подходит очень хорошо.  
- Прост в реализации и использовании.

<img src="data\sad-icon.png" alt="drawing" width="50"/>  

- Сложно настраивать под задачу из-за множества параметров.
- Нет гарантий сходимости.
- Выполнение может занимать много времени.

Метод отжига используется не очень часто, однако он довольно известен, и следует иметь о нём общее представление.

Если вам интересно подробнее узнать про метод отжига и посмотреть на его имплементацию в Python, рекомендуем ознакомиться со следующей статьёй.

https://dev.to/cesarwbr/how-to-implement-simulated-annealing-algorithm-in-python-4gid

### МЕТОД КООРДИНАТНОГО СПУСКА

Ещё один метод, про который необходимо узнать и с которым вы сталкивались ранее, — это **метод координатного спуска**.

Это тип алгоритма оптимизации, который обычно используется для «сильно выпуклого» случая регрессии — регрессионной функции Lasso и для регрессионной модели Elastic Net.

Метод координатного спуска можно считать одним из простейших методов оптимизации , который  в целом достаточно эффективно ищет локальные минимумы для гладких функций.

При поиске минимума с помощью этого метода мы всегда изменяем положение точки в направлении осей координат, т. е. всегда изменяется только одна координата, и благодаря этому задача становится одномерной.

Если визуализировать работу алгоритма, то мы увидим, что за счёт того, что каждый шаг происходит параллельно одной или другой координатной оси, ход алгоритма становится похож на «лесенку»:

<img src="data\MATHML_md6_7_2.png" alt="drawing" width="400"/>

По сути, мы просто выбираем некоторую точку и правило изменения координаты и движемся в соответствии с ним до локального минимума.

Посмотрим на **основные отличия координатного и градиентного спусков**:

<img src="data\PIC-3.png" alt="drawing" width="1100"/>

**ПРАКТИКА**

Предыдущий модуль мы завершили тем, что посмотрели на прописанный без готовых функций алгоритм классического градиентного спуска, чтобы построить прогностическую модель для продаж некоторого продукта на основе количества денег, потраченных на рекламу.

Пришло время попрактиковаться и попробовать самостоятельно реализовать алгоритмы, с которыми вы познакомились для решения той же задачи.

**Вам необходимо**:

1. Скачать файл с данными. https://lms.skillfactory.ru/assets/courseware/v1/be9ea3bcfb9e5ebc744c1f2af98fed61/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/Advertising.zip  
2. Открыть ноутбук-шаблон с заданием. https://lms.skillfactory.ru/assets/courseware/v1/7b2e6cbfd22f5452704aa9c75cff644b/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%D0%9E%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F.ipynb  
3. Загрузить данные и определить целевую переменную и предикторы.
4. Нормализовать признаки, с помощью которых вы будете осуществлять предсказание.
5. Реализовать координатный спуск, следуя подсказкам в файле-шаблоне.
6. Реализовать стохастический градиентный спуск, следуя подсказкам в файле-шаблоне.
7. Оценить качество обеих полученных моделей с помощью MSE и MAE.
8. Сделать содержательные выводы.

**Важно**:

- Вариант реализации алгоритмов, который будет предложен в подсказках, не единственно верный. Будет здорово, если вы сможете придумать полностью самостоятельную реализацию. Однако если будет сложно, можете воспользоваться нашим вариантом.
- Пожалуйста, оформите ваше решение в соответствии со следующими требованиями:
    - Решение оформляется только в Jupyter Notebook.
    - Решение оформляется в соответствии с ноутбуком-шаблоном.
    - Каждое задание выполняется в отдельной ячейке, выделенной под задание (в шаблоне они помечены как «ваш код здесь»). Не создавайте дополнительные ячейки.
    - Решение не должно содержать функции из библиотек для машинного обучения и оптимизации, кроме тех ячеек, где это указано явным образом.
    - Код должен быть читаемым и понятным: имена переменных и функций отражают их сущность, отсутствуют многострочные конструкции и условия.
        - Пользуйтесь руководством PEP-8.

**КРИТЕРИИ ОЦЕНИВАНИЯ**

Вам предстоит выполнить **три задания**, каждое из которых состоит из более мелких подзадач.

Для каждой подзадачи в ноутбуке-шаблоне указано количество баллов (если она оценивается отдельно). Один бонусный балл начисляется за аккуратное и полное выполнение всего задания.

**ЗАДАНИЕ	БАЛЛЫ**  
**1. Загрузка и подготовка данных**  
Аккуратное и полное выполнение задания	1  
**2. Координатный спуск**  
Реализация алгоритма координатного спуска	3  
Аккуратное и полное выполнение задания	1  
**3. Стохастический градиентный спуск**  
Масштабирование столбцов исходной матрицы	1  
Создание функции для вычисления среднеквадратичной ошибки	1  
Составление наивного прогноза (предсказание продаж средним значением)	1  
Создание функции для получения вектора прогнозов	1  
Создание функции для реализации шага стохастического градиентного спуска	2  
Создание функции для реализации стохастического градиентного спуска	3  
Аккуратное и полное выполнение задания	1  
Максимальное количество баллов — 15.  

Отправьте ноутбук (в формате .IPYNB) с выполненными заданиями на проверку через форму ниже ↓

ЗАДАНИЕ 7.4. МОДУЛЬ MATH&ML-6 (HW-02)